##### 版權 2024 Google LLC.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
# The non-source code materials on this page are licensed under Creative Commons - Attribution-ShareAlike CC-BY-SA 4.0,
# https://creativecommons.org/licenses/by-sa/4.0/legalcode.

# 使用 Gemini 嵌入進行搜尋重新排名


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/docs/search_reranking_using_embeddings"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />在 Google AI 上檢視</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/generative-ai-docs/blob/main/site/zh/docs/search_reranking_using_embeddings.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/doggy8088/generative-ai-docs/blob/main/site/zh/docs/search_reranking_using_embeddings.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />在 GitHub 上檢視原始碼</a>
  </td>
</table>


本筆記示範如何使用嵌入重新排名搜尋結果。本演練將著重於下列目標：



1.   設定你的開發環境和 API 存取權來使用 Gemini。
2.   使用 Gemini 的函式呼叫支援存取 Wikipedia API。
3.   透過 Gemini API 嵌入內容。
4.   重新排名搜尋結果。


這就是你將會實作搜尋重新排序的方法：




1.   使用者將會查詢模型。
2.   你將會使用 Wikipedia API 來回傳相關的搜尋結果。
3.   搜尋結果將會被嵌入，並且其相關性將會透過計算像是餘弦相似度、點積等距離度量來評估。
4.   最相關的結果將會被回傳為最後的答案。


## 先決條件

你可以在 [Google Colab](https://colab.research.google.com/github/doggy8088/generative-ai-docs/blob/main/site/zh/docs/search_reranking_using_embeddings.ipynb) 執行這個快速入門，它會直接在瀏覽器中執行此筆記本，而且不需要其他環境設定。


## 設定


Gemini API 的 Python SDK，包含於 [`google-generativeai`](https://pypi.org/project/google-generativeai/) 套件中。你還需要安裝 **Wikipedia** API。


In [ ]:
!pip install -q google-generativeai

In [ ]:
!pip install -q wikipedia

備註：[ `wikipedia` 套件](https://pypi.org/project/wikipedia/) 指出它「以簡單易用為設計，而非進階用途」，而生產或大量使用應「使用 [Pywikipediabot](http://www.mediawiki.org/wiki/Manual:Pywikipediabot) 或其他進階 [Python MediaWiki API 封裝](http://en.wikipedia.org/wiki/Wikipedia:Creating_a_bot#Python)」。


匯入必要的套件。


In [34]:
import json
import textwrap

import google.generativeai as genai
import google.ai.generativelanguage as glm

import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

import numpy as np

from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

### 取得 API 金鑰

在你開始使用 Gemini API 之前，首先必須取得 API 金鑰。如果你尚未取得，請在 Google AI Studio 中按一下即可建立金鑰。

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">取得 API 金鑰</a>


在 Colab 中，在左側面板的「🔑」下新增秘鑰至秘鑰管理服務。將其命名為 `GOOGLE_API_KEY`。


一旦你取得了 API 金鑰，請將其傳遞到 SDK。你可以用兩種方式進行此事：

* 將金鑰放入 `GOOGLE_API_KEY` 環境變數中 (SDK 會自動從中撷取)。
* 將金鑰傳遞到 `genai.configure(api_key=...)`


In [17]:
try:
    from google.colab import userdata
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
except ImportError:
    import os
    GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']

genai.configure(api_key=GOOGLE_API_KEY)

重點：接下來你將選擇一個模型。任何嵌入模型都適用於本教學示範，但對於實際應用來說，選擇特定模型，並堅持使用它非常重要。不同模型的輸出互不兼容。

**請注意** ：此時，Gemini API [僅在特定地區開放](https://developers.generativeai.google/available_regions)。


## 定義工具


如前所述，本教學使用 Gemini 的函式呼叫支援來存取維基百科 API。請參閱 [文件](https://ai.google.dev/docs/function_calling)，以進一步瞭解函式呼叫。


### 定義搜尋函式


為了迎合搜尋引擎的需求，你將以以下方式設計此功能：

* 在每個搜尋查詢中，搜尋引擎將使用 `wikipedia.search` 方法來取得相關主題。
* 從相關主題中，引擎將選擇 `n_topics(int)` 名候選人，並且將使用 `gemini-pro` 從頁面中擷取相關資訊。
* 引擎將透過維護搜尋歷程來避免重複項目。


In [41]:
def wikipedia_search(search_queries: list[str]) -> list[str]:
  """Search wikipedia for each query and summarize relevant docs."""
  n_topics=3
  search_history = set() # tracking search history
  search_urls = []
  mining_model = genai.GenerativeModel('gemini-pro')
  summary_results = []

  for query in search_queries:
    print(f'Searching for "{query}"')
    search_terms = wikipedia.search(query)

    print(f"Related search terms: {search_terms[:n_topics]}")
    for search_term in search_terms[:n_topics]: # select first `n_topics` candidates
      if search_term in search_history: # check if the topic is already covered
        continue

      print(f'Fetching page: "{search_term}"')
      search_history.add(search_term) # add to search history

      try:
        # extract the relevant data by using `gemini-pro` model
        page = wikipedia.page(search_term, auto_suggest=False)
        url = page.url
        print(f"Information Source: {url}")
        search_urls.append(url)
        page = page.content
        response = mining_model.generate_content(textwrap.dedent(f"""\
            Extract relevant information
            about user's query: {query}
            From this source:

            {page}

            Note: Do not summarize. Only Extract and return the relevant information
        """))

        urls = [url]
        if response.candidates[0].citation_metadata:
          extra_citations = response.candidates[0].citation_metadata.citation_sources
          extra_urls = [source.url for source in extra_citations]
          urls.extend(extra_urls)
          search_urls.extend(extra_urls)
          print("Additional citations:", response.candidates[0].citation_metadata.citation_sources)
        try:
          text = response.text
        except ValueError:
          pass
        else:
          summary_results.append(text + "\n\nBased on:\n  " + ',\n  '.join(urls))

      except DisambiguationError:
        print(f"""Results when searching for "{search_term}" (originally for "{query}")
        were ambiguous, hence skipping""")

      except PageError:
        print(f'{search_term} did not match with any page id, hence skipping.')

  print(f"Information Sources:")
  for url in search_urls:
    print('    ', url)

  return summary_results


In [42]:
example = wikipedia_search(["What are LLMs?"])

Searching for "What are LLMs?"
Related search terms: ['Large language model', 'Prompt engineering', 'Language model']
Fetching page: "Large language model"
Information Source: https://en.wikipedia.org/wiki/Large_language_model
Fetching page: "Prompt engineering"
Information Source: https://en.wikipedia.org/wiki/Prompt_engineering
Fetching page: "Language model"
Information Source: https://en.wikipedia.org/wiki/Language_model
Information Sources:
     https://en.wikipedia.org/wiki/Large_language_model
     https://en.wikipedia.org/wiki/Prompt_engineering
     https://en.wikipedia.org/wiki/Language_model


以下是搜尋結果的樣子：


In [43]:
from IPython.display import display

for e in example:
  display(to_markdown(e))

> **Relevant information about LLMs:**
> 
> * LLMs are language models notable for their ability to achieve general-purpose language generation and understanding.
> * LLMs are artificial neural networks, the largest and most capable of which are built with a decoder-only transformer-based architecture.
> * LLMs can be used for text generation, a form of generative AI, by taking an input text and repeatedly predicting the next token or word.
> * Some notable LLMs are OpenAI's GPT series of models, Google's PaLM and Gemini, Meta's LLaMA family of open-source models, and Anthropic's Claude models.
> * LLMs are trained using statistical relationships from text documents during a computationally intensive self-supervised and semi-supervised training process.
> * LLMs are thought to acquire knowledge about syntax, semantics, and "ontology" inherent in human language corpora, but also inaccuracies and biases present in the corpora.
> * LLMs can be used for a variety of tasks, including text generation, language translation, question answering, and summarization.
> * LLMs have a number of advantages over traditional language models, including their ability to handle longer sequences of text, their ability to learn from unlabeled data, and their ability to generate more coherent and fluent text.
> * LLMs also have a number of limitations, including their tendency to hallucinate facts, their lack of common sense knowledge, and their potential for bias.
> * LLMs are still under development, but they have the potential to revolutionize a wide range of industries, including natural language processing, customer service, and education.
> 
> Based on:
>   https://en.wikipedia.org/wiki/Large_language_model

> **LLMs** (Large Language Models) are powerful AI models that can understand and generate text. They are designed to perform various language-related tasks, such as answering questions, summarizing documents, translating languages, writing different forms of text, and generating code. LLMs have been significantly improved through techniques such as in-context learning, which allows them to temporarily learn from specific prompts.
> 
> **Prompt engineering** involves structuring text prompts to optimize the performance of an LLM. Effective prompts can guide the model's reasoning, provide context, and specify the desired output. Various prompt engineering techniques have been developed, including chain-of-thought prompting, generated knowledge prompting, and complexity-based prompting, each tailored to specific tasks and models.
> 
> LLMs have also been adapted to generate images and videos. **Text-to-image** models like DALL-E 2 create art based on textual descriptions, while **text-to-video** models generate videos from textual prompts. These models require specialized prompting techniques that account for their unique capabilities and limitations.
> 
> **Non-text prompts** are also used to guide LLMs. **Image prompting** allows users to provide images or image-based information as input, while **gradient descent**-based techniques enable the optimization of soft prompt tokens to enhance model performance.
> 
> **Prompt injection** is a security concern where malicious users craft prompts to trick LLMs into performing unintended actions or revealing sensitive information. Mitigation strategies include input filtering, output filtering, and prompt engineering techniques to separate user input from instructions.
> 
> LLMs are continually evolving, and new techniques and applications are being developed. They have the potential to revolutionize various industries by automating language-related tasks and enabling novel forms of creativity and communication.
> 
> Based on:
>   https://en.wikipedia.org/wiki/Prompt_engineering

> - Language models are probabilistic models of natural language.
> - Large language models are a combination of larger datasets, feedforward neural networks, and transformers.
> - Large language models are useful for a variety of tasks, including speech recognition, machine translation, natural language generation, optical character recognition, handwriting recognition, grammar induction, and information retrieval.
> - Evaluation of the quality of language models is mostly done by comparison to human-created sample benchmarks created from typical language-oriented tasks.
> 
> Based on:
>   https://en.wikipedia.org/wiki/Language_model

### 傳遞工具給該模型


若你傳遞函式清單給 `GenerativeModel` 的 `tools` 參數，
它會從函式特徵中擷取架構，並輸入提示，然後將架構傳遞到 API 呼叫。
對此，模型或許會回傳 `FunctionCall` 物件，要求呼叫函式。

注意：這種方法僅處理的註解為 `AllowedTypes = int | float | str | dict | list['AllowedTypes']`

`GenerativeModel` 會保留對函式本身的參照，以便它 _能_ 在稍後階段在本地執行函式。


In [45]:
model = genai.GenerativeModel(
    'gemini-pro',
    tools=[wikipedia_search],
    generation_config={'temperature': 0.6})

## 產生支援性搜尋查詢


為了讓用戶原本的查詢有更多支援的搜尋查詢，你會請模型產生更多這樣的查詢。這將幫助引擎在綜合層級涵蓋所詢問的問題。


In [46]:
instructions = """You have access to the Wikipedia API which you will be using
to answer a user's query. Your job is to generate a list of search queries which
might answer a user's question. Be creative by using various key-phrases from
the user's query. To generate variety of queries, ask questions which are
related to  the user's query that might help to find the answer. The more
queries you generate the better are the odds of you finding the correct answer.
Here is an example:

user: Tell me about Cricket World cup 2023 winners.

function_call: wikipedia_search(['What is the name of the team that
won the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup
2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What
was the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',
'Who lifted the Cricket World Cup 2023 trophy?'])

The search function will return a list of article summaries, use these to
answer the  user's question.

Here is the user's query: {query}
"""

為了産生更多有創意、更多樣化的發問，你將模型溫度參數設為一個較高的值。值可設在 [0.0,1.0] 內，兩端值包含在內。較接近 1.0 的值會産生更多樣化和極具創意的回應，而較接近 0.0 的值通常會産生更直白易懂的模型回應。

> 注意：從 [genai.GenerativeModel.GenerationConfig](https://ai.google.dev/api/python/google/generativeai/GenerationConfig) 類別中探索更多參數，以更好地控制模型的回應。


## 啟用自動函式呼叫並呼叫 API


現在使用 `enable_automatic_function_calling=True` 開始新的聊天。ChatGPT 將會處理呼叫函式所需的來回操作，並傳回最終回應：


In [47]:
model = genai.GenerativeModel(
    'gemini-pro', tools=[wikipedia_search], generation_config={'temperature': 0.6})

chat = model.start_chat(enable_automatic_function_calling=True)

query = "Explain how deep-sea life survives."

res = chat.send_message(instructions.format(query=query))

Searching for "How does deep-sea life survive?"
Related search terms: ['Deep sea', 'Deep-sea community', 'Hydrothermal vent']
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Fetching page: "Hydrothermal vent"
Information Source: https://en.wikipedia.org/wiki/Hydrothermal_vent
Searching for "What adaptations have deep-sea life developed to survive?"
Related search terms: ['Deep sea', 'Deep-sea fish', 'Deep-sea community']
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Searching for "What are the unique characteristics of deep-sea life?"
Related search terms: ['Deep-sea fish', 'Deep-sea community', 'Deep sea']
Searching for "What are the challenges deep-sea life faces?"
Related search terms: ['Deep sea', 'Marine habitat', 'Sea']
Fetching page: "Marine habitat"
Information Source: https://en.wikipedia.

In [48]:
to_markdown(res.text)

> Deep-sea life has evolved remarkable adaptations to survive the extreme conditions of the deep ocean. They have adapted to withstand high pressure, cold temperatures, and low oxygen levels. They have also developed unique ways to find food and communicate in the darkness of the deep sea.
> 
> Some of the adaptations of deep-sea life include:
> 
> * **High pressure tolerance:** Deep-sea organisms have evolved strong bodies to withstand the immense pressure of the deep ocean. Their bodies are often filled with a gelatinous substance that helps them to withstand the pressure.
> * **Cold tolerance:** Deep-sea organisms have adapted to the cold temperatures of the deep ocean. They have enzymes that function at low temperatures, and their bodies are often covered in a thick layer of insulation.
> * **Low oxygen tolerance:** Deep-sea organisms have adapted to the low oxygen levels of the deep ocean. They have evolved efficient respiratory systems that allow them to extract oxygen from the water.
> * ** Bioluminescence:** Many deep-sea organisms produce their own light, a process called bioluminescence. They use bioluminescence to attract prey, communicate with each other, and defend themselves from predators.
> * **Chemosynthesis:** Some deep-sea organisms do not rely on sunlight for food. Instead, they use a process called chemosynthesis to create food from chemicals in the water.
> 
> These are just a few of the adaptations that deep-sea life has evolved to survive in the extreme conditions of the deep ocean. These adaptations are a testament to the resilience and adaptability of life on Earth.

檢查額外的引文：


In [49]:
res.candidates[0].citation_metadata or 'No citations found'

'No citations found'

看來那奏效了。你可以瀏覽聊天記錄，查看在函式呼叫期間發送和接收內容的詳細資料：


In [52]:
for content in chat.history:
  part = content.parts[0]

  print(f'{content.role} -> ', end='')
  print(json.dumps(type(part).to_dict(part), indent=2))
  print('---' * 20)


user -> {
  "text": "You have access to the Wikipedia API which you will be using\nto answer a user's query. Your job is to generate a list of search queries which\nmight answer a user's question. Be creative by using various key-phrases from\nthe user's query. To generate variety of queries, ask questions which are\nrelated to  the user's query that might help to find the answer. The more\nqueries you generate the better are the odds of you finding the correct answer.\nHere is an example:\n\nuser: Tell me about Cricket World cup 2023 winners.\n\nfunction_call: wikipedia_search(['What is the name of the team that\nwon the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup\n2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What\nwas the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',\n'Who lifted the Cricket World Cup 2023 trophy?'])\n\nThe search function will return a list of article summaries, use these to\nans

在聊天記錄中，你可以看到全部 4 個步驟：

1. 使用者送出查詢。
2. 模型使用 `glm.FunctionCall` 回覆，呼叫 `wikipedia_search`，並提供數個相關搜尋。
3. 由於你在建立 `genai.ChatSession` 時啟用了 `enable_automatic_function_calling=True`，因此執行搜尋函式並將文章摘要清單傳回模型。
4. 模型依照提示中的說明，根據摘要產生最後答案。


## [選填] 手動執行函式呼叫


如果你想了解在背後發生的場景，此區段會手動執行`FunctionCall`來示範。


In [53]:
chat = model.start_chat()

In [54]:
result = chat.send_message(instructions.format(query=query))

一開始模型回傳一個函式呼叫：


In [65]:
fc = result.candidates[0].content.parts[0].function_call
fc = type(fc).to_dict(fc)
print(json.dumps(fc, indent=2))

{
  "name": "wikipedia_search",
  "args": {
    "search_queries": [
      "How do deep-sea animals survive?",
      "What are the adaptations of deep-sea creatures?",
      "How do deep-sea animals cope with extreme pressure?",
      "What are the unique characteristics of deep-sea organisms?",
      "How do deep-sea animals find food?",
      "How do deep-sea animals reproduce?",
      "What are the challenges faced by deep-sea animals?",
      "What is the role of deep-sea animals in the marine ecosystem?"
    ]
  }
}


In [66]:
fc['name']

'wikipedia_search'

使用產生的引數呼叫該函式以取得結果。


In [67]:
summaries = wikipedia_search(**fc['args'])

Searching for "How do deep-sea animals survive?"
Related search terms: ['Deep sea', 'Marine life', 'Deep-sea fish']
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Fetching page: "Marine life"
Information Source: https://en.wikipedia.org/wiki/Marine_life
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Searching for "What are the adaptations of deep-sea creatures?"
Related search terms: ['Deep sea', 'Deep-sea community', 'Deep-sea fish']
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Searching for "How do deep-sea animals cope with extreme pressure?"
Related search terms: ['Deep sea', 'Deep-sea fish', 'Deep-sea community']
Searching for "What are the unique characteristics of deep-sea organisms?"
Related search terms: ['Deep-sea fish', 'Deep sea', 'Deep-sea community']
Searching for "How do deep-sea animals find food?"
Related search terms: ['Deep

現在把 `FunctionResult` 傳送給模型。


In [68]:
response = chat.send_message(
    glm.Content(
      parts=[glm.Part(
          function_response = glm.FunctionResponse(
            name='wikipedia_search',
            response={'result': summaries}
          )
      )]
    )
)

to_markdown(response.text)

> **Deep-sea life survives by adapting to the extreme conditions of the deep ocean, including high pressure, low temperatures, and lack of light.**
> 
> **Adaptations for survival include:**
> 
> * **High internal pressure:** Deep-sea animals have high internal pressure that matches the external pressure, preventing them from being crushed.
> * **Buoyancy adaptations:** Many deep-sea fish have a gelatinous layer below the skin or around the spine for buoyancy and swimming efficiency. They also have low tissue density, achieved through high fat content, reduced skeletal weight, and water accumulation, allowing them to float without a swim bladder.
> * **Light and vision:** Deep-sea fish lack sunlight, so they rely on other senses, such as sensitivity to pressure changes and smell, for locating prey and mates. Many deep-sea fish are bioluminescent, using light to communicate, attract prey, or camouflage themselves. Some have sensitive eyes with high numbers of Rh1 genes, helping them see in low light conditions.
> * **Feeding mechanisms:** Deep-sea fish often have large mouths and sharp teeth for consuming prey of similar or larger sizes. They use feelers to locate prey in the darkness.
> * **Behavior:** Mesopelagic fish make vertical migrations following zooplankton prey, returning to deeper depths during the day. Bathypelagic fish are sedentary, waiting for prey to come close enough or being lured by bioluminescence. Some deep-sea fish are hermaphrodites, increasing their chances of reproduction in the sparse environment.
> * **Physiological adaptations:** Deep-sea animals have slow metabolisms and unspecialized diets, allowing them to survive with limited food availability. Their proteins are structurally modified to withstand high pressure, ensuring enzymatic reactions and cellular processes function properly. Na+/K+ -ATPase, involved in osmoregulation, is more tolerant of pressure in deep-sea fish compared to shallow-water species.

## 重新排名搜尋結果


協助函式來嵌入內容：


In [69]:
def get_embeddings(content: list[str]) -> np.ndarray:
  embeddings = genai.embed_content('models/embedding-001', content, 'SEMANTIC_SIMILARITY')
  embds = embeddings.get('embedding', None)
  embds = np.array(embds).reshape(len(embds), -1)
  return embds

請參考[嵌入式指南](https://ai.google.dev/docs/embeddings_guide)取得更多有關嵌入式的資訊


下一步是定義可用於計算兩個嵌入向量之間相似度分數的函式。這些分數將協助你決定哪個嵌入向量與使用者的查詢最相關。


你現在將實作**餘弦相似度** 作為衡量標準。在此，返回的嵌入向量將是單位長度，因此它們的 L1 範數 (`np.linalg.norm()`) 將為 ~1。因此，計算**餘弦相似度** 與計算它們的**點積分數** 本質上是相同的。


In [70]:
def dot_product(a: np.ndarray, b: np.ndarray):
  return (a @ b.T)

### 與使用者的查詢相似度

現在是時候找到由維基百科 API 傳回的最相關搜尋結果。

使用 Gemini API 來為使用者的查詢和搜尋結果取得嵌入碼。


In [71]:
search_res = get_embeddings(summaries)
embedded_query = get_embeddings([query])

計算相似度得分：


In [72]:
sim_value = dot_product(search_res, embedded_query)

使用 `np.argmax` 選擇最佳候選。

**使用者輸入：** 解釋深海生物如何生存。

**回答：** 


In [73]:
print(summaries[np.argmax(sim_value)])

In this document, there is no information about how deep-sea animals survive.

Based on:
  https://en.wikipedia.org/wiki/Marine_life


### 與假設文件嵌入 (HyDE) 的相似性

從 [[Gao et al](https://arxiv.org/abs/2212.10496)] 汲取靈感，此處的目標是使用 Gemini Pro 的內部知識為使用者的查詢產生範本答案。此假設答案將作為計算所有搜尋結果相關性的基準。


In [74]:
hypothetical_ans_model = genai.GenerativeModel('gemini-pro')
res = hypothetical_ans_model.generate_content(f"""Generate a hypothetical answer
to the user's query by using your own knowledge. Assume that you know everything
about the said topic. Do not use factual information, instead use placeholders
to complete your answer. Your answer should feel like it has been written by a human.

query: {query}""")

to_markdown(res.text)

> In the enigmatic depths where sunlight surrenders to inky blackness, life perseveres, illuminated by the faintest of luminescent flickers.
> 
> Imagine a realm where extreme pressure could crush the mightiest of vessels. Yet, in this unforgiving abyss, creatures have evolved with bodies resilient as the very seafloor. Their flexible exoskeletons or gelatinous tissues withstand the crushing weight gracefully.
> 
> Oxygen, a lifeline for most creatures, grows scarce with depth. Enter our deep-sea dwellers, whose bodies have ingeniously adapted. They absorb oxygen directly through their skin or gills, maximizing every molecule they find.
> 
> Nutrient scarcity plagues these depths, where sunlight cannot penetrate to foster photosynthesis. Instead, these creatures rely on chemosynthesis, a remarkable process that utilizes chemicals from hydrothermal vents or decaying matter.
> 
> In the perpetual darkness, vision becomes obsolete. Instead, sensory organs have evolved to detect minute vibrations, bioluminescence, and heat gradients, guiding them through the shadowy labyrinth.
> 
> Temperature fluctuations can be drastic, from freezing cold to scalding heat. But deep-sea creatures have mastered the art of thermoregulation, their internal systems finely tuned to withstand the extremes.
> 
> Growth is a slow and arduous process in these unforgiving depths. Many species exhibit extreme longevity, surviving for centuries or even millennia. Their life cycles are meticulously paced, ensuring their survival in this harsh environment.
> 
> Reproduction is a perilous task, with offspring often vulnerable and exposed. Some deep-sea creatures protect their young with parental care, nurturing them until they can fend for themselves in this unforgiving realm.
> 
> The deep sea, a testament to the resilience and adaptability of life, is a fascinating and mysterious world. Its inhabitants continue to inspire awe and wonder, reminding us of the extraordinary diversity and ingenuity that exists within our planet's watery depths.

使用 Gemini API 為基線回答取得嵌入，並將它們與搜尋結果比較


In [75]:
hypothetical_ans = get_embeddings([res.text])

計算相似度評分以對搜尋結果進行排名


In [76]:
sim_value = dot_product(search_res, hypothetical_ans)

In [77]:
sim_value

array([[0.72687077],
       [0.73694087],
       [0.77235092],
       [0.75185433],
       [0.63363508],
       [0.62639701],
       [0.71418557],
       [0.70211815]])

使用 `np.argmax` 選擇最佳候選。

**使用者輸入：** 解釋深海生物如何生存。

**回答：** 


In [78]:
to_markdown(summaries[np.argmax(sim_value)])

> **How do deep-sea animals survive?**
> 
> **Adaptations to Pressure:**
> 
> * Deep-sea animals have high internal pressure that matches the external pressure, preventing them from being crushed.
> * Their cell membranes contain a higher proportion of unsaturated fatty acids, which increases membrane fluidity in high-pressure environments.
> 
> **Buoyancy Adaptations:**
> 
> * Many deep-sea fish have a gelatinous layer below the skin or around the spine for buoyancy and swimming efficiency.
> * They have low tissue density, achieved through high fat content, reduced skeletal weight, and water accumulation, allowing them to float without a swim bladder.
> 
> **Light and Vision:**
> 
> * Deep-sea fish lack sunlight, so they rely on other senses, such as sensitivity to pressure changes and smell, for locating prey and mates.
> * Many deep-sea fish are bioluminescent, using light to communicate, attract prey, or camouflage themselves.
> * Some have sensitive eyes with high numbers of Rh1 genes, helping them see in low light conditions.
> 
> **Feeding Mechanisms:**
> 
> * Deep-sea fish often have large mouths and sharp teeth for consuming prey of similar or larger sizes.
> * They use feelers to locate prey in the darkness.
> 
> **Behavior:**
> 
> * Mesopelagic fish make vertical migrations following zooplankton prey, returning to deeper depths during the day.
> * Bathypelagic fish are sedentary, waiting for prey to come close enough or being lured by bioluminescence.
> * Some deep-sea fish are hermaphrodites, increasing their chances of reproduction in the sparse environment.
> 
> **Physiological Adaptations:**
> 
> * Deep-sea animals have slow metabolisms and unspecialized diets, allowing them to survive with limited food availability.
> * Their proteins are structurally modified to withstand high pressure, ensuring enzymatic reactions and cellular processes function properly.
> * Na+/K+ -ATPase, involved in osmoregulation, is more tolerant of pressure in deep-sea fish compared to shallow-water species.
> 
> Based on:
>   https://en.wikipedia.org/wiki/Deep-sea_fish

你現在已利用嵌入式建立搜尋重新排名引擎！


## 後續步驟

如需瞭解如何使用 Gemini API 中的其他服務，請參閱 [Python 快速入門](https://ai.google.dev/tutorials/python_quickstart)。如需瞭解有關如何使用字向量表示的更多資訊，請查看下列可用的 [範例](https://ai.google.dev/examples?keywords=embed)。
